In [1]:
#Importing necessary libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd

#Base uel for Iphone8 search on ebay
url ='https://www.ebay.com/sch/i.html?_from=R40&_trksid=p2380057.m570.l1312.R1.TR10.TRC2.A0.H0.Xi+ph.TRS2&_nkw=i+phone+8&_sacat=0'
#Emtpy Dictionary to store each item's details
item_dict = {}
#Empyt list to store all the items
item_list = []
#Looping through 20 pages
for i in range(20):
    page_url = url + '&_pgn=' + str(i)

    res = requests.get(page_url)
    
    # Raises an exception error if there's an error downloadPing the website    
    res.raise_for_status()
    
    # Creates a BeautifulSoup object for HTML parsing
    soup = BeautifulSoup(res.text, 'html.parser')
    #Getting each item details into results 
    results = soup.find_all('div', {"class":  "s-item__info clearfix"} )
    for result in results:
        # Scrapes the listed item's name
        name = result.find("h3", {"class": "s-item__title"}).get_text(separator=u" ")
        # Scrapes the listed item's price
        price = result.find("span", {"class": "s-item__price"}).get_text()
        # Scrapes the listed item's shipping details
        shipping = result.find("span", {"class": "s-item__shipping s-item__logisticsCost"}).get_text()
        # Scrapes the listed item's free return details
        #returns = result.find("span", {"class": "s-item__free-returns s-item__freeReturnsNoFee"}).get_text()
        # Scrapes the listed item's review count
        #reviews_count = result.find("span", {"class": "s-item__reviews-count"}).get_text()
        # Scrapes the listed item's type (pre-owned, refurbished etc)
        item_type = result.find("span", {"class": "SECONDARY_INFO"}).get_text()
        
        #INserting individual item details into the dictionary
        item_dict['Item Name'] = name        
        item_dict['Item Price'] = price
        item_dict['Shipping'] = shipping
        #item_dict['Returns'] = returns
        #item_dict['Review Count'] = reviews_count
        item_dict['Item Type'] = item_type
        
        #Appending into the list
        
        item_list.append(dict(item_dict))
        

In [2]:
#Creating DataFRaem out of the list of Dictionaries
item_df = pd.DataFrame(item_list)
item_df.head()

,Item Name,Item Price,Shipping,Item Type
0,Apple iPhone 8 - 64GB-256GB GSM/CDMA Unlocked ...,$299.00,Free Shipping,Refurbished
1,Apple iPhone 8 Smartphone AT&T Sprint T-Mobile...,$250.00 to $389.00,Free Shipping,Refurbished
2,Apple iPhone 8 - 64GB 256GB Smartphone Unlocke...,$299.99 to $429.99,Free Shipping,Refurbished
3,Apple iPhone 8 64GB-256GB GSM Unlocked A1905 –...,$309.00,Free Shipping,Refurbished
4,Apple iPhone 8 64GB | T-Mobile Network | - F...,$274.95 to $314.95,Free Shipping,Refurbished


In [3]:
#Import Sqlalchemy to establish connection and to do db operations
from sqlalchemy import create_engine
connection_string = "postgres:Satasreea123^@localhost:5432/IPhone8"
engine = create_engine(f'postgresql://{connection_string}')
#Check tables in DB
engine.table_names()

['item_detail']

In [4]:
#Insert DataFrame data into DB
item_df.to_sql(name='item_detail', con=engine, if_exists='append', index=False)

In [5]:
pd.read_sql_query('select count(*) from item_detail', con=engine).head()

,count
0,1549


In [9]:
#Query to see top Iphone 8 Prices
pd.read_sql_query('select * from item_detail order by 3,5 desc', con=engine).head(15)

,id,Item Name,Item Price,Shipping,Item Type
0,9106,Apple iPhone 8 - 64GB - Space Gray (Unlocked) ...,"$1,200.00",Free Shipping,Refurbished
1,9110,Apple Iphone 8,$100.00,Free Shipping,Pre-Owned
2,9198,New Listing Apple iPhone 7 - 128GB - Black (Un...,$100.00,+$12.90 shipping,Pre-Owned
3,8487,New Listing Apple iPhone 8 Plus - 256GB - Spac...,$100.00,Free Shipping,Pre-Owned
4,8524,Apple iPhone 8 - 64GB - Space Gray (Sprint) A1...,$100.00,+$9.30 shipping,Pre-Owned
5,8543,Apple iPhone 8 - 64GB - Space Gray (Sprint) A1...,$100.00,+$9.30 shipping,Pre-Owned
6,8467,New Listing Apple iPhone 8 Plus - 256GB - Spac...,$100.00,Free Shipping,Pre-Owned
7,7991,New Listing Apple iPhone 8 - 256GB - Space Gra...,$102.50,Free Shipping,Pre-Owned
8,8058,Apple iPhone 8 - 64GB - Silver (Verizon) A1863...,$107.50,+$9.45 shipping,Pre-Owned
9,8594,New Listing Apple iPhone 8 - 256GB - Space Gra...,$11.50,+$9.30 shipping,Pre-Owned
